In [ ]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime, date, timedelta

In [ ]:
import sqlite3

conn = sqlite3.connect("../EXOPLANETS.db")
c = conn.cursor()

In [ ]:
def show_query_plot(filename, x, y):
    with open(filename, 'r') as sql_file:
        sql = sql_file.read()

    # '.headers on' works for command line execution but causes errors in the python environment
    data = pd.read_sql_query(re.sub('.headers on', '', sql), conn)
    data = data.rename(columns={clmn: clmn.lower() for clmn in data.columns})
    return px.bar(data, x=x, y=y)

In [ ]:
def show_plot_with_detections(detections_file, data_file, x, y):
    fig = show_query_plot(data_file, x, y)

    with open(detections_file, 'r') as sql_file:
        sql = sql_file.read()
    detections = pd.read_sql_query(re.sub('.headers on', '', sql), conn)
    detections = detections.rename(columns={clmn: clmn.lower() for clmn in detections.columns})
    for _, row in detections.iterrows():
        fig.add_vline(x=row['date_added'], line_color='red')
    
    return fig

In [ ]:
freshness_query = """WITH UPDATES AS(
  SELECT
    DATE_ADDED,
    COUNT(*) AS ROWS_ADDED
  FROM
    EXOPLANETS
  GROUP BY
    DATE_ADDED
),

NUM_DAYS_UPDATES AS (
  SELECT
    DATE_ADDED,
    JULIANDAY(DATE_ADDED) - JULIANDAY(LAG(DATE_ADDED)
      OVER(
        ORDER BY DATE_ADDED
      )
    ) AS DAYS_SINCE_LAST_UPDATE
  FROM
    UPDATES
)

SELECT
  *
FROM
  NUM_DAYS_UPDATES
WHERE
  DAYS_SINCE_LAST_UPDATE > {threshold_days};"""

In [ ]:
def show_freshness_detections(threshold):
    fig = show_query_plot('../queries/freshness/rows-added.sql', 'date_added', 'rows_added')

    detections = pd.read_sql_query(freshness_query.format(threshold_days=threshold), conn)
    detections = detections.rename(columns={clmn: clmn.lower() for clmn in detections.columns})
    for _, row in detections.iterrows():
        fig.add_vline(x=row['date_added'], line_color='red')
    
    return fig

In [ ]:
show_freshness_detections(1).show()

In [ ]:
show_freshness_detections(3).show()

In [ ]:
show_freshness_detections(7).show()

In [ ]:
show_query_plot('../queries/freshness/days-since-last-update.sql', 'date_added', 'days_since_last_update').show()

In [ ]:
show_plot_with_detections(
    '../queries/freshness/freshness-detector.sql',
    '../queries/freshness/rows-added.sql',
    'date_added',
    'rows_added'
).show()

In [ ]:
show_query_plot('../queries/distribution/null-rates.sql', 'date_added', 'distance_null_rate').show()

In [ ]:
show_query_plot('../queries/distribution/null-rates.sql', 'date_added', 'g_null_rate').show()

In [ ]:
show_query_plot('../queries/distribution/null-rates.sql', 'date_added', 'orbital_period_null_rate').show()

In [ ]:
show_query_plot('../queries/distribution/null-rates.sql', 'date_added', 'avg_temp_null_rate').show()

In [ ]:
show_plot_with_detections(
    '../queries/distribution/naive-dist-detector.sql',
    '../queries/distribution/null-rates.sql',
    'date_added',
    'avg_temp_null_rate'
).show()

In [ ]:
show_plot_with_detections(
    '../queries/distribution/dist-query-only-start-dates.sql',
    '../queries/distribution/null-rates.sql',
    'date_added',
    'avg_temp_null_rate'
).show()